# Deep MNIST para Expertos


En esta libreta realizaremos el tutorial de esta [pagina](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html).

TensorFlow es una libreria poderosa capaz de realizar operaciones a gran escala. Una de las tareas en las que sobresale es en implementar y entrenar redes neuronales profundas.

Se vera la construccion basica para construir un modelo clasificador profundo MNIST.

# Setup

Antes de que creemos nuestro modelo debemos de cargar los datos del MNIST y empezar una sesion del TensorFlow.

# Se cargan los datos MNIST 

Utilizaremos las siguientes lineas para leer los datos del MNIST_data.


In [1]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Comenzar una Sesion TensorFlow

TensorFlow trabaja com c++ para realizar los calculos. esta conexion se llama session. 

Vamos a utilizar la clase **InteractiveSession**, la cual hace a TensorFlow mas flexible acerca de como estructurar el codigo. permitiendo intervalos de operaciones con intervalos de computo grafico.

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

# Construir un modelo de regresion Softmax

En esta seccion vamos a construir un model softmax con una sola capa lineal, despues vamos a agregar mas capas para crear una red.

# Placeholders

Comenzamos construyendo el grafo computacional creando nodos para las imagenes de entrada y para las classes de salida.



In [3]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

**x** y **y_** son placeholder para que el TensorFlow llene  cuando realize los calculos.

# Variables

Ahora definiremos los pesos **W** y el bias **b** para nuestro modelo. Vamos a utlizar una funcion llamada Variable la cual puede ser usada o modificada por los calculos.

In [4]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Antes de que las variables puedan ser utilizadas se debe de inicializar una sesion.

In [5]:
sess.run(tf.initialize_all_variables())

# Clase Predict y Funcion de Coste

Ahora podemos implementar nuestro modelo de regresion.

In [6]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

La funcion de costo que debe minimizarse durante el entrenamiento se puede especificar facilmente. Nuestra funcion de costo va a se el cross-entropy entre nuestro objetivo y la prediccion.

In [7]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

Nota que **tf.reduce_sum** suma atravez de todas las imaagenes en un minibatch, asi como en todas las clases.

# Entrenar el Modelo

Ahora que hemos definido nuestro modelo y la funcion de costo del entrenamiento, esto se hace facilmente con el TensorFlow.

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

Lo que realizo TensorFlow con una sola linea fue agregar nuevas operaciones de computo, estas incluyen las que calculan el gradiente. las que calculan los parametros que se actualizara, y la que aplica estas actualizaciones a los parametros.

La operacion **train_step** va a aplicar el cambio en los parametros del decenso de gradiente. Por lo tanto el entrenamiento del modelo puede ser acompletado realizando esta operacion varias veces.

In [9]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

En Cada iteracion de entrenamiento cargamos 50 ejemplos. despues corremos **train_step** utilizando  **feed_dict** para remplazar los placeholder que tenemos con los ejemplos de entenamiento.

# Evalua el Modelo

Vamos a utilizar **tf.argmax** la cual es una funcion extremadamente util ya que nos da el indice de la entrada mas grande. Podemos usar **tf.equal** para checar si nuestras predicciones son iguales a la verdad.

In [10]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

Esto nos da una lista de booleanos. Ahora utilizamos **tf.reduce_mean** para obtener el porcentaje de correctas.

In [11]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Por ultimo podemos evaluar nuestra precision con los datos de prueba. esto nos deberia dar 91% aproximadamente.

In [12]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


# Construir una Red Multicapa

Obtener una precision del 91% es malo. Ahora vamos a pasar a un modelo sofisticado, una red neuronal. esto nos dara una precision del 99.2%.

# Inicializacion de pesos

para crear este modelo vamos a tener que crear muchos pesos y biases. Siempre se deben de inicializar los pesos con cantidades pequeñas de ruido y prevenir gradientes de 0. Como usaremos neuronas ReLU, es buena practica inicializar con una bias un poca positiva para evitar neuronas muertas.
Ahora haremos dos funciones muy utiles.

In [13]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# Convolution y Pooling

TensorFlow tambien nos da mucha flexivilidad en operaciones de convolution y pooling.

In [14]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Primera capa

Ahora podemos implementar la primera capa que consiste de convolution seguido de max pooling. La forma que tendra los pesos sera de la forma **[5, 5, 1, 32]**. los primeros dos digitos son para el tamaño, el siguiente es el numero de canales de entrada, y el ultimo es el numero de canales de salida.

In [15]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

Para aplicar la capa primero cambiamos la forma de **x** a 4d tensor, donde la segunda y tercer dimension son el tamaño de la imagen, y el ultimo es el numero de color del canal.

In [16]:
x_image = tf.reshape(x, [-1,28,28,1])

Despues convolve **x_image** con el peso, agregamos la bias, aplicamos la funcion ReLU y por ultimo el max pool.

In [17]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Segunda Capa

En orden de construir una red profunda, ponemos varias capas de este tipo. la segunda capa tendra 64 features.

In [18]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Capa Densamente Conectada

Ahora que el tamaño de la imagen a sido reducido a 7x7 agregamos una capa con 1024 neuronas para poder procesar la imagen entera.

In [19]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout

para reducir sobreaprendizaje, vamos a salir antes de la capa readout. creamos un **placeholder** por si ocupamos la probabilidad de la neurona de salida si hacemos un dropout.
TensorFlow **tf.nn.dropout** automaticamente se encarga de escalar las salidas de las neuronas.

In [20]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Capa Readout

Finalmente agregamos una capa de softmax, la cual es igual a la regresion softmax anterior.

In [21]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# Entrena y Evalua el Modelo

Para entrenar y evaluar se va a utilizar un codigo casi identico al de softmax. la diferencia es que se va a reemplazar el decenso de gradiente por una forma pas sofisticada. vamos a incluir parametros adicionales para controlar la frecuencia de dropout y vamos a imprimir cada 100 iteraciones.

In [22]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.02
step 100, training accuracy 0.8
step 200, training accuracy 0.9
step 300, training accuracy 0.9
step 400, training accuracy 0.94
step 500, training accuracy 0.92
step 600, training accuracy 0.9
step 700, training accuracy 1
step 800, training accuracy 0.96
step 900, training accuracy 0.9
step 1000, training accuracy 0.96
step 1100, training accuracy 0.9
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 0.94
step 1500, training accuracy 1
step 1600, training accuracy 0.94
step 1700, training accuracy 1
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.94
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 0.94
step 2400, training accuracy 0.98
step 2500, training accuracy 0.98
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, train

ResourceExhaustedError: OOM when allocating tensor with shapedim { size: 10000 } dim { size: 28 } dim { size: 28 } dim { size: 32 }
	 [[Node: add_1 = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Conv2D, Variable_3)]]
Caused by op u'add_1', defined at:
  File "/home/umu/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/umu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/umu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/umu/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/umu/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/home/umu/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 151, in 

El test final deberia ser aproximadamente 99.2%.